In [2]:
import numpy as np
import pandas as pd
np.random.seed(42)
N = 50
T = 200
n_sectors = 10
n_factors = 12
seeds = list(range(1, 20))

factor_scores = np.random.randn(N, n_factors)
mcap = np.random.uniform(0.5, 10.0, size=N)
index_caps = mcap / mcap.sum()
sectors = np.random.choice(range(n_sectors), size=N)
alpha = 0.95
scale = 1.0 / ((1 - alpha) * T)

def compute_cvar_np(w, R):
    L = -R.dot(w)
    var = np.percentile(L, alpha * 100)
    return var + scale * np.maximum(L - var, 0).sum()

results=[]
for seed in seeds:
    np.random.seed(seed)
    drifts=np.random.uniform(0.0005,0.002,size=N)
    vols=np.random.uniform(0.015,0.03,size=N)
    eps=np.random.randn(T,N)
    log_rets=(drifts-0.5*vols**2)+vols*eps
    returns=np.exp(log_rets)-1
    w=index_caps
    mean_daily=returns.dot(w).mean()
    sharpe=mean_daily/returns.dot(w).std(ddof=1)*np.sqrt(252)
    cum_ret=np.prod(1+returns.dot(w))-1
    cvar95=compute_cvar_np(w,returns)
    results.append({'Seed':seed,'MeanDaily':mean_daily,'Sharpe':sharpe,'Cumulative':cum_ret,'CVaR95':cvar95})

print(pd.DataFrame(results).describe().loc[['mean','std','min','max']])

           Seed  MeanDaily    Sharpe  Cumulative    CVaR95
mean  10.000000   0.001185  5.043623    0.267280  0.006293
std    5.627314   0.000278  1.203574    0.071207  0.000642
min    1.000000   0.000702  3.057482    0.149226  0.005443
max   19.000000   0.001771  7.896371    0.422762  0.007522


In [3]:
import numpy as np
import pandas as pd
import cvxpy as cp

# Synthetic data setup
np.random.seed(42)
N = 50
T = 200
n_sectors = 5
n_factors = 5
seeds = list(range(1, 20))

factor_scores = np.random.randn(N, n_factors)
mcap = np.random.uniform(0.5, 10.0, size=N)
index_caps = mcap / mcap.sum()
sectors = np.random.choice(range(n_sectors), size=N)
# allow up to 30% per sector to ensure feasibility of the
# non-negativity and index-weight constraints
sector_cap = 0.3

alpha = 0.95
target_cvar = 0.02
scale = 1.0 / ((1 - alpha) * T)


def compute_cvar_np(w, R):
    L = -R.dot(w)
    var = np.percentile(L, alpha * 100)
    return L[L >= var].mean()


def compute_sortino_np(w, R, req=0.0):
    r = R.dot(w)
    excess = r - req
    dd = np.sqrt(np.mean(np.minimum(excess, 0)**2))
    return (excess.mean() / dd * np.sqrt(252)) if dd > 0 else np.nan


def compute_max_dd_np(w, R):
    W = np.cumprod(1 + R.dot(w))
    peak = np.maximum.accumulate(W)
    return ((peak - W) / peak).max()

results = []
for seed in seeds:
    np.random.seed(seed)
    drifts = np.random.uniform(0.0005, 0.002, size=N)
    vols = np.random.uniform(0.015, 0.03, size=N)
    eps = np.random.randn(T, N)
    log_rets = (drifts - 0.5 * vols**2) + vols * eps
    returns = np.exp(log_rets) - 1

    premia = []
    for f in range(n_factors):
        r = np.clip(factor_scores[:, f], 0, None)
        w0 = np.minimum(r, index_caps)
        w0 /= w0.sum()
        premia.append(w0 @ factor_scores[:, f])
    top2 = np.argsort(premia)[-2:]
    mu = factor_scores[:, top2].sum(axis=1)
    mu = (mu - mu.mean()) / mu.std(ddof=1)

    w = cp.Variable(N)
    tau = cp.Variable()
    z = cp.Variable(T)

    eps = 1e-6  # small slack to ensure a non-empty interior
    cons = [
        cp.sum(w) == 1,
        w >= 0,
        w <= index_caps + eps
    ]
    for s in range(n_sectors):
        idx = np.where(sectors == s)[0]
        if idx.size:
            cons.append(cp.sum(w[idx]) <= sector_cap)
    losses = -returns @ w
    cons += [
        z >= 0,
        z >= losses - tau,
        tau + scale * cp.sum(z) <= target_cvar
    ]

    prob = cp.Problem(cp.Maximize(mu @ w), cons)
    prob.solve(solver=cp.ECOS_BB, verbose=False)
    w_opt = w.value

    mean_daily = returns.dot(w_opt).mean()
    sharpe = mean_daily / returns.dot(w_opt).std(ddof=1) * np.sqrt(252)
    sortino = compute_sortino_np(w_opt, returns)
    cum_ret = np.prod(1 + returns.dot(w_opt)) - 1
    cvar95 = compute_cvar_np(w_opt, returns)
    max_dd = compute_max_dd_np(w_opt, returns)

    results.append({
        "Seed": seed,
        "MeanDaily": mean_daily,
        "Sharpe": sharpe,
        "Sortino": sortino,
        "Cumulative": cum_ret,
        "CVaR95": cvar95,
        "MaxDrawdown": max_dd
    })


if __name__ == "__main__":
    df = pd.DataFrame(results)
    print("ECOS_BB Optimized Portfolio Summary (across seeds):")
    print(df.describe().loc[["mean", "std", "min", "max"]].to_string())

TypeError: unsupported operand type(s) for *: 'float' and 'NoneType'